In [1]:
with open("./inputs/input.txt") as file:
    lines = file.readlines()

In [2]:
wharehouse = [l.strip("\n") for l in lines]

In [3]:
GUARD_STEPS = {
    ">": (0, 1),
    "<": (0, -1),
    "^": (-1, 0),
    "v": (1, 0),
}

GUARD_TURNS = {
    ">": "v",
    "<": "^",
    "^": ">",
    "v": "<",
}


def replace_char(string: str, char: str, index: int) -> str:
    if len(char) > 1:
        raise ValueError("char can only be of length 1")
    return string[:index] + char + string[index + 1 :]


def check_next_step(wharehouse: list[str], next_coord: tuple[int, int], marker: str = "#") -> bool:
    next_step = wharehouse[next_coord[0]][next_coord[1]]
    if next_step == marker:
        return True

    return False


def turn_guard(wharehouse: list[str], coord: tuple[int, int]) -> list[str]:
    guard = wharehouse[coord[0]][coord[1]]
    turned_guard = GUARD_TURNS[guard]
    wharehouse[coord[0]] = replace_char(wharehouse[coord[0]], turned_guard, coord[1])

    return wharehouse


def take_step(
    wharehouse: list[str], coord: tuple[int, int], new_coord: tuple[int, int], path_marker: str = "X"
) -> list[str]:
    guard = wharehouse[coord[0]][coord[1]]
    wharehouse[coord[0]] = replace_char(wharehouse[coord[0]], path_marker, coord[1])
    wharehouse[new_coord[0]] = replace_char(wharehouse[new_coord[0]], guard, new_coord[1])

    return wharehouse


def get_potential_next_position(wharehouse: list[str], coord: tuple[int, int]) -> tuple[int, int]:
    guard = wharehouse[coord[0]][coord[1]]
    step = GUARD_STEPS[guard]
    return coord[0] + step[0], coord[1] + step[1]


def get_guard_position(wharehouse: list[str]) -> tuple[int, int]:
    guard_list = [">", "<", "^", "v"]
    for i, l in enumerate(wharehouse):
        for g in guard_list:
            j = l.find(g)
            if j != -1:
                return i, j


def get_next_wharehouse_layout(wharehouse: list[str], border_marker: str = "o") -> tuple[list[str], bool]:
    exit_loop = False
    guard_coord = get_guard_position(wharehouse)
    next_coord = get_potential_next_position(wharehouse, guard_coord)

    if check_next_step(wharehouse, next_coord):
        return turn_guard(wharehouse, guard_coord), exit_loop

    if check_next_step(wharehouse, next_coord, border_marker):
        exit_loop = True

    new_wharehouse_layout = take_step(wharehouse, guard_coord, next_coord)

    return new_wharehouse_layout, exit_loop


def pad_wharehouse_layout(wharehouse: list[str], border_marker: str = "o") -> list[str]:
    padded_layout = []
    width_wharehouse = len(wharehouse[0])
    top_bottom_padding = border_marker * (width_wharehouse + 2)
    padded_layout.append(top_bottom_padding)
    for l in wharehouse:
        padded_layout.append(border_marker + l + border_marker)
    padded_layout.append(top_bottom_padding)
    return padded_layout


def compute_path_length(wharehouse: list[str], path_marker: str = "X") -> int:
    path_length = 0
    for l in wharehouse:
        path_length += l.count(path_marker)

    return path_length

# Puzzle 1

In [4]:
wharehouse = pad_wharehouse_layout(wharehouse)

while True:
    wharehouse, exit_loop = get_next_wharehouse_layout(wharehouse)
    if exit_loop:
        break

path_length = compute_path_length(wharehouse)

In [5]:
print(f"{path_length=}")

path_length=5080


In [6]:
wharehouse

['oooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooo',
 'o......#........#.............#.##..........................................#.......#...#........#......................#..........o',
 'o.......................#...#................................#..#.........................................#.....#.....#............o',
 'o.......................................#.................#..................#....#........................................#.......o',
 'o.............#...................#.#...............................#....#....#.#.......................................#..#.......o',
 'o...............................#........##.#..............................#..............................#..#...#..#..............o',
 'o........................#......XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX#..#...............o',
 'o............................#..X......